# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Introdução à Inteligência Artificial</font>

## Probabilidade e Redes Bayesianas

A teoria da probabilidade nos permite calcular a probabilidade de certos eventos, considerando pressupostos sobre os componentes do evento. Uma rede bayesiana, ou rede de Bayes para abreviar, é uma estrutura de dados para representar uma distribuição de probabilidade conjunta sobre diversas variáveis aleatórias, e fazer inferência sobre ela.

Como exemplo, aqui está uma rede com cinco variáveis aleatórias, cada uma com sua tabela de probabilidade condicional, e com setas das variáveis pai a filho. A história é que há um alarme doméstico contra assaltante, que pode ser disparado por um assalto ou um terremoto. Se o alarme soar, um ou ambos os vizinhos, John e Mary, podem chamar (telefonar) o dono da casa para dizer que o alarme está soando.

<p><img src="burglary.jpg">

Implementamos isso com a ajuda de sete classes Python:


## `BayesNet()`

Um `BayesNet` é um gráfico (como no diagrama acima) onde cada nó representa uma variável aleatória, e as arestas são links filho. Você pode construir um gráfico vazio com `BayesNet()`, então adicionar as variáveis uma de cada vez com a chamada de método `.add (` * nome_da_variável, nomes_de_mãe, cpt * `)`, onde os nomes são strings e cada um dos `Parent_names` já deve ter sido` .add`ed.

## `Variable(`*name, cpt, parents*`)`

Uma variável aleatória; As elipses no diagrama acima. O valor de uma variável depende do valor dos pais, de uma maneira probabilística especificada pela tabela de probabilidade condicional (CPT) da variável. Dados os pais, a variável é independente de todas as outras variáveis. Por exemplo, se eu sei que * Alarme * é verdadeiro ou falso, então eu sei a probabilidade de * JohnCalls *, e a evidência sobre as outras variáveis não me dará mais informações sobre * JohnCalls *. Cada linha do CPT usa a mesma ordem de variáveis que a lista de pais.
Só vamos permitir variáveis com um domínio finito discreto; Não valores contínuos.

## `ProbDist(`*mapping*`)`<br>`Factor(`*mapping*`)`

Uma distribuição de probabilidade é um mapeamento de `{resultado: probabilidade}` para cada resultado de uma variável aleatória. Você pode dar `ProbDist` os mesmos argumentos que você daria para o ` dict` inicializador, por exemplo
`ProbDist (sol = 0.6, chuva = 0.1, nublado = 0.3)`.
Como atalho para Variáveis Booleanas, você pode dizer `ProbDist (0.95)` em vez de `ProbDist ({T: 0.95, F: 0.05})`.
Em uma distribuição de probabilidade, cada valor está entre 0 e 1, e os valores somam 1.
Um `Factor` é semelhante a uma distribuição de probabilidade, exceto que os valores não precisam somar a 1. Fatores
São utilizados no método de inferência de eliminação variável.

## `Evidence(`*mapping*`)`

Um mapeamento dos pares `{Variable: value, ...}`, descrevendo os valores exatos para um conjunto de variáveis - as coisas que sabemos com certeza.

## `CPTable(`*rows, parents*`)`

Uma tabela de probabilidade condicional (ou * CPT *) descreve a probabilidade de cada possível valor de resultado de uma variável aleatória, dados os valores das variáveis-pai. A `CPTable` é a mapping,` {tuple: probdist, ...} `, onde cada tupla lista os valores de cada uma das variáveis pai, por ordem, e cada distribuição de probabilidade diz quais são os resultados possíveis, dados esses valores dos pais. O `CPTable` para * Alarm * no diagrama acima seria representado da seguinte forma:

    CPTable({(T, T): .95,
             (T, F): .94,
             (F, T): .29,
             (F, F): .001},
            [Burglary, Earthquake])
            
Como você lê isso? Pegue a segunda linha, "` (T, F): .94` ". Isto significa que quando o primeiro pai (`Burglary`) é verdadeiro, e o segundo pai (` Earthquake`) é falso, então a probabilidade de `Alarm` ser verdadeira é .94. Note que o .94 é uma abreviatura de `ProbDist ({T: .94, F: .06})`.
            
## `T = Bool(True); F = Bool(False)`

Quando usei valores `bool` (` True` e `False`), tornou-se difícil ler as linhas em CPTables, porque as colunas não se alinhavam:

     (True, True, False, False, False)
     (False, False, False, False, True)
     (True, False, False, True, True)
     
Portanto, criei a classe `Bool`, com constantes` T` e `F`, tais que` T == True` e `F == False`, e agora as linhas são mais fáceis de ler:

     (T, T, F, F, F)
     (F, F, F, F, T)
     (T, F, F, T, T)
     
Aqui está o código para essas classes:

In [1]:
from collections import defaultdict, Counter
import itertools
import math
import random

class BayesNet(object):
    "Rede bayesiana: um gráfico de variáveis conectadas por links pai."
     
    def __init__(self): 
        self.variables = [] # Lista de variáveis, na ordem de classificação topológica dos pais
        self.lookup = {}    # Mapeamento de pares {variable_name: variable}
            
    def add(self, name, parentnames, cpt):
        "Adicione uma nova Variável ao BayesNet. Os nomes dos pais devem ter sido adicionados anteriormente."
        parents = [self.lookup[name] for name in parentnames]
        var = Variable(name, cpt, parents)
        self.variables.append(var)
        self.lookup[name] = var
        return self
    
class Variable(object):
    "Uma variável aleatória discreta; Condicional em zero ou mais variáveis pai."
    
    def __init__(self, name, cpt, parents=()):
        "Uma variável tem um nome, uma lista de variáveis pai e uma Tabela de Probabilidade Condicional."
        self.__name__ = name
        self.parents  = parents
        self.cpt      = CPTable(cpt, parents)
        self.domain   = set(itertools.chain(*self.cpt.values())) 
                
    def __repr__(self): return self.__name__
    
class Factor(dict): "An {outcome: frequency} mapping."

class ProbDist(Factor):
    """A distribuição de probabilidade é um mapeamento {outcome: probabilidade}.
     Os valores são normalizados para somar a 1.
     ProbDist (0.75) é uma abreviatura para ProbDist ({T: 0.75, F: 0.25})."""
    def __init__(self, mapping=(), **kwargs):
        if isinstance(mapping, float):
            mapping = {T: mapping, F: 1 - mapping}
        self.update(mapping, **kwargs)
        normalize(self)
        
class Evidence(dict): 
    "Um mapeamento {variable: value}, descrevendo o que sabemos com certeza."
        
class CPTable(dict):
    "Um mapeamento de {row: ProbDist, ...} onde cada linha é uma tupla de valores das variáveis pai."
    
    def __init__(self, mapping, parents=()):
        """Fornece dois atalhos para escrever uma Tabela de Probabilidade Condicional.
         Sem pais, CPTable (dist) significa CPTable ({(): dist}).
         Com um pai, CPTable ({val: dist, ...}) significa CPTable ({(val,): dist, ...})."""
        if len(parents) == 0 and not (isinstance(mapping, dict) and set(mapping.keys()) == {()}):
            mapping = {(): mapping}
        for (row, dist) in mapping.items():
            if len(parents) == 1 and not isinstance(row, tuple): 
                row = (row,)
            self[row] = ProbDist(dist)

class Bool(int):
    "Assim como `bool`, exceto os valores exibidos como 'T' e 'F' em vez de 'True' e 'False'"
    __str__ = __repr__ = lambda self: 'T' if self else 'F'
        
T = Bool(True)
F = Bool(False)

E aqui estão algumas funções associadas:

In [2]:
def P(var, evidence={}):
    "A distribuição de probabilidade para P (variável | evidência), quando todas as variáveis-mãe são conhecidas (em evidência)."
    row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row]

def normalize(dist):
    "Normalize uma distribuição {key: value} para que os valores somem a 1.0. Mutates dist e retorna-lo."
    total = sum(dist.values())
    for key in dist:
        dist[key] = dist[key] / total
        assert 0 <= dist[key] <= 1, "As probabilidades devem estar entre 0 e 1."
    return dist

def sample(probdist):
    "Amostragem aleatória de um resultado de uma distribuição de probabilidade."
    r = random.random() # r é um ponto aleatório na distribuição de probabilidade
    c = 0.0             # c é a probabilidade cumulativa de resultados vistos até agora
    for outcome in probdist:
        c += probdist[outcome]
        if r <= c:
            return outcome
        
def globalize(mapping):
    "Dado um mapeamento {name: value}, exporte todos os nomes para o espaço de nomes `globals ()`."
    globals().update(mapping)

# Exemplo

Aqui estão alguns exemplos de como usar as classes:

In [3]:
# Exemplo de variável aleatória: Terremoto:
# Um terremoto ocorre em 0,002 dias, independente de qualquer outra variável.
Earthquake = Variable('Earthquake', 0.002)

In [4]:
# A distribuição de probabilidade para o terremoto
P(Earthquake)

{T: 0.002, F: 0.998}

In [5]:
# Obter a probabilidade de um resultado específico, subscrito a distribuição de probabilidade
P(Earthquake)[T]

0.002

In [6]:
# Amostragem aleatória da distribuição:
sample(P(Earthquake))

F

In [7]:
# Amostragem aleatória 100.000 vezes, e contar os resultados:
Counter(sample(P(Earthquake)) for i in range(100000))

Counter({F: 99779, T: 221})

In [8]:
# Duas formas equivalentes de especificar a mesma distribuição de probabilidade booleana:
assert ProbDist(0.75) == ProbDist({T: 0.75, F: 0.25})

In [9]:
# Duas formas equivalentes de especificar a mesma distribuição de probabilidade não booleana:
assert ProbDist(win=15, lose=3, tie=2) == ProbDist({'win': 15, 'lose': 3, 'tie': 2})
ProbDist(win=15, lose=3, tie=2)

{'win': 0.75, 'lose': 0.15, 'tie': 0.1}

In [10]:
# A diferença entre um Factor e um ProbDist - o ProbDist é normalizado:
Factor(a=1, b=2, c=3, d=4)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [11]:
ProbDist(a=1, b=2, c=3, d=4)

{'a': 0.1, 'b': 0.2, 'c': 0.3, 'd': 0.4}

# Exemplo: Alarme Bayes Net

Eis como definimos a rede de Bayes a partir do diagrama acima:

<img src="burglary.jpg">

In [12]:
alarm_net = (BayesNet()
    .add('Burglary', [], 0.001)
    .add('Earthquake', [], 0.002)
    .add('Alarm', ['Burglary', 'Earthquake'], {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001})
    .add('JohnCalls', ['Alarm'], {T: 0.90, F: 0.05})
    .add('MaryCalls', ['Alarm'], {T: 0.70, F: 0.01}))  

In [13]:
# Faça de Roubo, Terremoto, etc., variáveis globais
globalize(alarm_net.lookup) 
alarm_net.variables

[Burglary, Earthquake, Alarm, JohnCalls, MaryCalls]

In [14]:
# Distribuição de probabilidade de um roubo
P(Burglary)

{T: 0.001, F: 0.999}

In [15]:
# Probabilidade de Alarme disparando, dado um Roubo e não um Terremoto:
P(Alarm, {Burglary: T, Earthquake: F})

{T: 0.94, F: 0.06000000000000005}

In [16]:
# De onde veio: a linha (T, F) do CPT do alarme:
Alarm.cpt

{(T, T): {T: 0.95, F: 0.050000000000000044},
 (T, F): {T: 0.94, F: 0.06000000000000005},
 (F, T): {T: 0.29, F: 0.71},
 (F, F): {T: 0.001, F: 0.999}}

# Bayes Nets como distribuições conjuntas de probabilidade

Uma rede de Bayes é uma maneira compacta de especificar uma distribuição conjunta completa sobre todas as variáveis na rede. Dado um conjunto de variáveis, a distribuição completa da articulação é: (x1)

P(*X*<sub>1</sub>=*x*<sub>1</sub>, ..., *X*<sub>*n*</sub>=*x*<sub>*n*</sub>) = <font size=large>&Pi;</font><sub>*i*</sub> P(*X*<sub>*i*</sub> = *x*<sub>*i*</sub> | parents(*X*<sub>*i*</sub>))

Para uma rede com variáveis * n *, cada uma das quais tem valores de * b *, há linhas * b <sup> n </ sup> * na distribuição conjunta (por exemplo, um bilhão de linhas para 30 variáveis booleanas) É impraticável criar explicitamente a distribuição conjunta para grandes redes. Mas para redes pequenas, a função `joint_distribution` cria a distribuição, que pode ser instrutiva e pode ser usada para fazer inferência.

In [17]:
def joint_distribution(net):
    "Dada uma rede de Bayes, crie a distribuição conjunta sobre todas as variáveis."
    return ProbDist({row: prod(P_xi_given_parents(var, row, net)
                               for var in net.variables)
                     for row in all_rows(net)})

def all_rows(net): return itertools.product(*[var.domain for var in net.variables])

def P_xi_given_parents(var, row, net):
    "A probabilidade de que var = xi, dados os valores nesta linha."
    dist = P(var, Evidence(zip(net.variables, row)))
    xi = row[net.variables.index(var)]
    return dist[xi]

def prod(numbers):
    "O produto dos números: prod ([2, 3, 5]) == 30. Analogamente a `sum ([2, 3, 5]) == 10`."
    result = 1
    for x in numbers:
        result *= x
    return result

In [18]:
# Todas as linhas na distribuição conjunta (2 ** 5 == 32 linhas)
set(all_rows(alarm_net))

{(F, F, F, F, F),
 (F, F, F, F, T),
 (F, F, F, T, F),
 (F, F, F, T, T),
 (F, F, T, F, F),
 (F, F, T, F, T),
 (F, F, T, T, F),
 (F, F, T, T, T),
 (F, T, F, F, F),
 (F, T, F, F, T),
 (F, T, F, T, F),
 (F, T, F, T, T),
 (F, T, T, F, F),
 (F, T, T, F, T),
 (F, T, T, T, F),
 (F, T, T, T, T),
 (T, F, F, F, F),
 (T, F, F, F, T),
 (T, F, F, T, F),
 (T, F, F, T, T),
 (T, F, T, F, F),
 (T, F, T, F, T),
 (T, F, T, T, F),
 (T, F, T, T, T),
 (T, T, F, F, F),
 (T, T, F, F, T),
 (T, T, F, T, F),
 (T, T, F, T, T),
 (T, T, T, F, F),
 (T, T, T, F, T),
 (T, T, T, T, F),
 (T, T, T, T, T)}

In [19]:
# Vamos trabalhar com apenas uma linha da tabela:
row = (F, F, F, F, F)

In [20]:
# Esta é a distribuição de probabilidade para Alarme
P(Alarm, {Burglary: F, Earthquake: F})

{T: 0.001, F: 0.999}

In [21]:
# Aqui está a probabilidade de que Alarme seja falso, dados os valores pai nesta linha:
P_xi_given_parents(Alarm, row, alarm_net)

0.999

In [22]:
# A distribuição conjunta completa:
joint_distribution(alarm_net)

{(F, F, F, F, F): 0.9367427006190001,
 (F, F, F, F, T): 0.009462047481000001,
 (F, F, F, T, F): 0.04930224740100002,
 (F, F, F, T, T): 0.0004980024990000002,
 (F, F, T, F, F): 2.9910060000000004e-05,
 (F, F, T, F, T): 6.979013999999999e-05,
 (F, F, T, T, F): 0.00026919054000000005,
 (F, F, T, T, T): 0.00062811126,
 (F, T, F, F, F): 0.0013341744900000002,
 (F, T, F, F, T): 1.3476510000000005e-05,
 (F, T, F, T, F): 7.021971000000001e-05,
 (F, T, F, T, T): 7.092900000000001e-07,
 (F, T, T, F, F): 1.7382600000000002e-05,
 (F, T, T, F, T): 4.0559399999999997e-05,
 (F, T, T, T, F): 0.00015644340000000006,
 (F, T, T, T, T): 0.00036503460000000007,
 (T, F, F, F, F): 5.631714000000006e-05,
 (T, F, F, F, T): 5.688600000000006e-07,
 (T, F, F, T, F): 2.9640600000000033e-06,
 (T, F, F, T, T): 2.9940000000000035e-08,
 (T, F, T, F, F): 2.8143600000000003e-05,
 (T, F, T, F, T): 6.56684e-05,
 (T, F, T, T, F): 0.0002532924000000001,
 (T, F, T, T, T): 0.0005910156000000001,
 (T, T, F, F, F): 9.4050000000

In [23]:
# Probabilidade de que "o alarme soou, mas nem um roubo nem um terremoto ocorreu,
# E tanto John e Mary fizeram a chamada 

print(alarm_net.variables)
joint_distribution(alarm_net)[F, F, T, T, T]

[Burglary, Earthquake, Alarm, JohnCalls, MaryCalls]


0.00062811126

# Inferência por Consulta da Distribuição Conjunta

Podemos usar `P(variável, evidência)` para obter a probabilidade de uma variável a, se conhecemos todas as variáveis pai. Mas e se nós não soubermos? As redes Bayes permitem calcular a probabilidade, mas o cálculo não é apenas uma pesquisa no CPT; É um cálculo global em toda a rede. Uma forma ineficiente mas simples de fazer o cálculo é criar a distribuição de probabilidade conjunta, e então selecionar apenas as linhas que correspondem as variáveis de evidência e, para cada linha, verificar qual é o valor da variável de consulta e incrementar a probabilidade para esse valor de acordo:

In [24]:
def enumeration_ask(X, evidence, net):
    "A distribuição de probabilidade para a variável de consulta X em uma rede de crenças,."
    i    = net.variables.index(X) 
    dist = defaultdict(float)    
    for (row, p) in joint_distribution(net).items():
        if matches_evidence(row, evidence, net):
            dist[row[i]] += p
    return ProbDist(dist)

def matches_evidence(row, evidence, net):
    "A tupla de valores para esta linha concorda com a evidência?"
    return all(evidence[v] == row[net.variables.index(v)]
               for v in evidence)

In [25]:
# A probabilidade de um Roubo, dado que João chama, mas Maria não:
enumeration_ask(Burglary, {JohnCalls: F, MaryCalls: T}, alarm_net)

{F: 0.9931237539265789, T: 0.006876246073421024}

In [26]:
# A probabilidade de um Alarme, dado que há um Terremoto e Maria chama:
enumeration_ask(Alarm, {MaryCalls: T, Earthquake: T}, alarm_net)

{F: 0.03368899586522123, T: 0.9663110041347788}

## Fim